In [8]:
import time
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd

In [3]:

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784) / 255.0
X_test = X_test.reshape(-1, 784) / 255.0
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [4]:

def build_model():
    return Sequential([
        Dense(256, activation='relu', input_shape=(784,)),
        Dense(256, activation='relu'),
        Dense(10, activation='softmax')
    ])

In [5]:

def benchmark(device):
    with tf.device(device):
        model = build_model()
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        start = time.time()
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        train_time = time.time() - start
        _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return train_time, accuracy

In [6]:

cpu_time, cpu_acc = benchmark('/CPU:0')
gpu_time, gpu_acc = benchmark('/GPU:0') if tf.config.list_physical_devices('GPU') else (None, None)

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:

results = pd.DataFrame({
    'Device': ['CPU', 'GPU'],
    'Training Time (s)': [cpu_time, gpu_time],
    'Test Accuracy': [cpu_acc, gpu_acc]
})
print(results)

print("\nNote: GPU typically provides 5-10x speedup for matrix")
print("operations in deep networks, though exact improvement")
print("depends on hardware and batch sizes.")

  Device  Training Time (s)  Test Accuracy
0    CPU          75.121817         0.9762
1    GPU                NaN            NaN

Note: GPU typically provides 5-10x speedup for matrix
operations in deep networks, though exact improvement
depends on hardware and batch sizes.
